# Bllossom-8B 양자화 모델을 이용한 한국어 RAG

In [1]:
!CMAKE_ARGS="-DGGML_BLAS=ON -DGGML_BLAS_VENDOR=OpenBLAS" pip install --upgrade --verbose --force-reinstall --no-cache-dir llama-cpp-python #양자화 구동을 위한 Llama C++ 설치
# conda install -c conda-forge gcc gxx
# !huggingface-cli download MLP-KTLim/llama-3-Korean-Bllossom-8B-gguf-Q4_K_M --local-dir='./' #Bllossom모델 다운로드

In [ ]:
from llama_cpp import Llama
import transformers
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch

model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

pipeline.model.eval()

In [2]:
from transformers import AutoModel, AutoTokenizer
import json
from datetime import datetime
import chromadb

COLLECTION_NAME = "cnu_post_data"
COLLECTION_NAME2 = "cnu_post_content_data"

client = chromadb.PersistentClient()
collection = client.get_or_create_collection(name=COLLECTION_NAME)
collection2 = client.get_or_create_collection(name=COLLECTION_NAME2)

model = AutoModel.from_pretrained('BM-K/KoSimCSE-roberta')
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-roberta')

In [ ]:
single_query = '이번학기 수강신청 언제야?'

documents = list()
documents.append(single_query)

inputs = tokenizer(documents, padding=True, truncation=True, return_tensors="pt")
embeddings, _ = model(**inputs, return_dict=False)
doc_embeddings = []

for embedded_doc in embeddings:
    doc_embedding = embedded_doc[0].detach().cpu().numpy().tolist()
    doc_embeddings.append(doc_embedding)

LIMIT = 15

query_result = collection2.query(
            query_embeddings=doc_embeddings,
            where={"학과명": {"$in": ["컴퓨터융합학부","인공지능학과"]}},
            n_results=LIMIT)

for idx,result_documents in enumerate(query_result["documents"][0]):
        print(idx,result_documents)

In [3]:
import json
from datetime import datetime

with open('golden_dataset.json', 'r',encoding='utf-8') as file:
    dataset = json.load(file)
    
# # 모든 데이터 삽입
for data in dataset:
    documents = list()
    query = data["question"]
    documents.append(query)

    inputs = tokenizer(documents, padding=True, truncation=True, return_tensors="pt")
    embeddings, _ = model(**inputs, return_dict=False)
    doc_embeddings = []

    for embedded_doc in embeddings:
        doc_embedding = embedded_doc[0].detach().cpu().numpy().tolist()
        doc_embeddings.append(doc_embedding)


    query_result = collection2.query(
            query_embeddings=doc_embeddings,
            where={"학과명": {"$in": ["컴퓨터융합학부","인공지능학과"]}},
            n_results=15
        )




    PROMPT = \
        f'''You are a helpful AI assistant. You must respond to user queries logicaly, kindly and accurately in korean.
        Refer to the searched documents to answer the user query. retrieved documents: {query_result["documents"][0]}
        If the information from the retrieved documents is insufficient, respond with "Insufficient information available." and
        When a document have a time information, you must mention it correctly, Today is {datetime.now()}
        '''

    instruction = f'{query} '

    messages = [
            {"role": "system", "content": f"{PROMPT}"},
            {"role": "user", "content": f"{instruction}"}
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
    )

    generation_kwargs = {
            "max_tokens":1024,
            "stop":["<|eot_id|>"],
            "top_p":0.9,
            "temperature":0.6,
            "echo":True, # Echo the prompt in the output
    }

    terminators = [
            pipeline.tokenizer.eos_token_id,
            pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]

    outputs = pipeline(
            prompt,
            max_new_tokens=2048,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9
        )

    # Answer
    answer = outputs[0]["generated_text"][len(prompt):]

    data.update({"answer": answer,
                    "retrieved_documents": query_result["documents"][0],
                    "context":PROMPT})

    print(answer)

    with open('generated_dataset.json','a',encoding='utf-8') as gfile:
            json.dump(data,gfile,ensure_ascii=False)
            gfile.write(',\n')
    

    # 90m 16s

수강 신청 취소는 본 수강신청 기간 동안 수강신청 화면에서 수강신청 내역에 있는 과목 중 앞에 '삭제' 버튼을 누르면 취소됩니다. 

또한, 수강 신청 취소 기간 동안만 취소가 가능하며, 취소 기간 이후에는 취소가 불가하므로 신중히 주시기 바랍니다.

2024년 하기 계절학기 수강신청 취소 기간은 다음과 같습니다:

1. 1차 수강신청 취소: 2024년 6월 11일(화) ~ 6월 13일(목)
2. 2차 수강신청 취소: 2024년 6월 27일(목) ~ 6월 28일(금)
3. 3차 수강신청 취소: 2024년 7월 1일(월) ~ 7월 2일(화)

취소 기간 동안 학생은 포털 로그인 후 통합정보시스템에서 학사행정 > 수강정보 > 수강취소 메뉴를 통해 취소할 과목의 '취소신청' 버튼을 클릭하여 취소 신청할 수 있습니다. 취소 후 '수강신청내역/시간표'에서 2024년 4월 2일 이후로 수강신청 내용이 삭제됩니다.

또한, 수강료 납부 후 수강 신청을 취소하고자 하는 경우, 납부한 수강료는 환불 대상이 됩니다. '0'원 납부자(예: 사회봉사 등)는 은행을 방문하여 고지서 제출 및 납부해야 합니다.

학사학위취득 유예 신청은 학기 개시 후 2주 이내에 가능하며, 당초 졸업예정 시기에 졸업 처리됩니다. 학사학위취득 유예를 취소하고자 하는 경우, 학사학위취득 유예 허가를 받은 후에는 취소가 불가합니다. 다만, 사망, 군입대 등 학과장이 학업을 계속할 수 없다고 인정하는 경우에는 취소 신청서를 제출하여 학사학위취득 유예를 취소할 수 있습니다.

기타 문의사항은 학사지원과(042-821-5042)로 문의하시기 바랍니다.
2024학년도 1학기 인공지능학과 및 컴퓨터융합학부의 일부 교과목에서 학부생 튜터를 선발합니다. 활동 기간은 2024년 3월 1일부터 2024년 6월 23일까지입니다. 많은 학생들의 관심과 신청을 바랍니다.

### 지원 신청 일정 및 방법

#### 가. 담당교수님께 튜터 추천 신청

- 기한: 2024년 2월 23일(금) 정오까지
- 신청 방법: 교과목 담당 교

In [5]:
import json

with open('generated_dataset.json','r',encoding='utf-8') as gfile:
    gdataset = json.load(gfile)


for idx, gdata in enumerate(gdataset):
    question = gdata['question']
    answer = gdata['answer']
    retrived = gdata['retrieved_documents']
    print(f"Q-{idx}.",question)
    print(f"A-{idx}. ",answer)
    print(f"Retrived-{idx} [")
    for ridx, rdoc in enumerate(retrived):
        print(f"[{ridx}]",rdoc)
    print(f"]")
    print("----------------------")

    

Q-0. 이번 학기 수강신청 언제야?
A-0.  2024학년도 하기 계절학기 수강신청은 다음과 같습니다:

가. 수업기간: 2024. 6. 24.(월) ~ 7. 12.(금) [월 · 화 · 수 · 목 · 금]

나. 수강신청기간: 2024. 5. 9.(목) 09:30 ~ 5. 13.(월) 18:00 [3일간]

다. 수강신청방법: 수강신청시스템 (https://sugang.cnu.ac.kr) 또는 모바일 수강신청 앱

라. 유의사항: 수강신청 (변경) 기간 이후 추가 수강신청은 불가하므로 졸업 등 사유로 계절학기 수강이 필요한 학생은 기간 내에 신청해야 합니다.
Retrived-0 [
[0] 2024 학년도 하기 계절학기 수강신청 을 다음과 같이 안내드립니다.  가 . 수업기간 (3 주 , 15 일 ) : 2024. 6. 24.( 월 ) ~ 7. 12.( 금 )  [ 월 · 화 · 수 · 목 · 금 ]  나 . 수강신청기간 : 2024. 5. 9.( 목 ) 09:30 ~ 5. 13.( 월 ) 18:00  [3 일간 ]  다 . 수강신청방법 : 수강신청시스템 ( https://sugang.cnu.ac.kr ) 또는 모바일 수강신청 앱  라 . 유의사항 : 수강신청 ( 변경 ) 기간 이후 추가 수강신청 은 불가 하므로 졸업 등 사유로 계절 학기 수강이 필요한 학생은 반드시 기간 내 신청 붙임 1. 2024 학년도 하기 계절학기 수강신청 안내문 1 부 .  2. 2024 학년도 하기 계절학기 학사일정표 1 부 .  3. 2024 학년도 하기 계절학기 수업시간표 1 부 . .
[1] 2024 학년도 하기 계절학기 수강신청 을 다음과 같이 안내드립니다.  가 . 수업기간 (3 주 , 15 일 ) : 2024. 6. 24.( 월 ) ~ 7. 12.( 금 )  [ 월 · 화 · 수 · 목 · 금 ]  나 . 수강신청기간 : 2024. 5. 9.( 목 ) 09:30 ~ 5. 13.( 월 ) 18:00  [3 일간 ]  다 . 수강신청방법 : 수강신청시스템 ( https://

In [6]:
import json

with open('golden_dataset.json','r',encoding='utf-8') as gfile:
    golden_dataset = json.load(gfile)


for idx, golden_data in enumerate(golden_dataset):
    question = golden_data['question']
    answer = golden_data['answer']

    print(f"Q-{idx}.",question)
    print(f"A-{idx}. ",answer)

    print("----------------------")

Q-0. 이번 학기 수강신청 언제야?
A-0.  이번 학기 수강신청 일정은 다음과 같습니다:

### 공학인증대상학생
- **재학생 (1학년)**: 2024년 8월 1일 (목) 09:30 ~ 12:00
- **재학생 (1, 2, 3, 4, 5학년)**: 2024년 8월 1일 (목) 14:00 ~ 17:30

### 학부생
- **4, 5학년**: 2024년 8월 5일 (월) 09:30 ~ 11:30
- **3학년**: 2024년 8월 5일 (월) 15:00 ~ 17:00
- **2학년**: 2024년 8월 6일 (화) 09:30 ~ 11:30
- **1학년**: 2024년 8월 6일 (화) 15:00 ~ 17:00
- **전체 학년 (1, 2, 3, 4, 5학년)**: 2024년 8월 7일 (수) ~ 8월 9일 (금) 09:00 ~ 18:00

### 대학원생
- **전학년**: 2024년 8월 5일 (월) ~ 8월 9일 (금) 09:00 ~ 18:00

이 일정을 참고하셔서 수강신청 준비 잘 하세요!
----------------------
Q-1. 백마 인턴십에 대해서 알려줘
A-1.  백마 인턴십은 충남대학교와 다양한 산업체가 공동으로 참여하여 학생들에게 실무 교육과 실습 기회를 제공하는 산학협력 교육과정입니다. 백마 인턴십의 주요 내용은 다음과 같습니다:

### 참여 대상
- 4학기 이상 이수한 학생
- 학기별 요건이 다를 수 있으므로 반드시 해당 운영 계획서를 확인해야 합니다.

### 실습 시간
- 주 40시간(1일 8시간) 이내에서 연속적으로 실시

### 교과 정보
백마 인턴십은 계절제와 학기제로 구분됩니다.

#### 계절제
1. **백마인턴십 III**
   - 학점: 3
   - 이수 구분: 일반 선택
   - 최소 실습 시간: 4주(160시간 이상)
   - 운영 시기: 
     - 여름: 6월 말 ~ 8월
     - 겨울: 12월 말 ~ 차년도 2월

2. **백마인턴십 I**
   - 학점: 6
   - 이수 구분: 일반 선택
 

In [10]:

PROMPT = \
    f'''You are a helpful AI assistant. You must respond to user queries logicaly, kindly and accurately in korean.
    Refer to the searched documents to answer the user query. retrieved documents: {query_result["documents"][0]}
    If the information from the retrieved documents is insufficient, respond with "Insufficient information available." and
    When a document have a time information, you must mention it correctly, Today is {datetime.now()}
    '''

instruction = f'{query} 하기계절은 알려주지마'

messages = [
        {"role": "system", "content": f"{PROMPT}"},
        {"role": "user", "content": f"{instruction}"}
]

prompt = pipeline.tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
)

generation_kwargs = {
        "max_tokens":1024,
        "stop":["<|eot_id|>"],
        "top_p":0.9,
        "temperature":0.6,
        "echo":True, # Echo the prompt in the output
}

terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

outputs = pipeline(
        prompt,
        max_new_tokens=2048,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )

# Answer
answer = outputs[0]["generated_text"][len(prompt):]

data.update({"answer": answer,
                "retrieved_documents": query_result["documents"][0],
                "context":PROMPT})

print(answer)

with open('generated_dataset.json','a',encoding='utf-8') as gfile:
        json.dump(data,gfile,ensure_ascii=False)
        gfile.write(',\n')

2024학년도 하기 계절학기 수강신청은 다음과 같이 안내드립니다.

가. 수업기간 (3주, 15일) : 2024. 6. 24.(월) ~ 7. 12.(금) [월 · 화 · 수 · 목 · 금]

나. 수강신청기간 : 2024. 5. 9.(목) 09:30 ~ 5. 13.(월) 18:00 [3일간]

다. 수강신청방법 : 수강신청시스템 (https://sugang.cnu.ac.kr) 또는 모바일 수강신청 앱

라. 유의사항 : 수강신청 (변경) 기간 이후 추가 수강신청은 불가하므로, 졸업 등 사유로 계절학기 수강이 필요한 학생은 반드시 기간 내에 신청하시기 바랍니다.
